In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dataset_dir = '/content/drive/My Drive/scopus/' #directory of all folders with the abstracts
 #directory must be structured like:
 # /scopus/
 #        /AI_scopus.csv
 #        /data_scopus



Mounted at /content/drive


Use GPU:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import nltk

nltk.download("book")
### Check if a cuda GPU is available
if torch.cuda.is_available():
    print('GPU availble')
    # Define the device (here you can select which GPU to use if more than 1)
    device = torch.device("cuda")
else:
    print('GPU not availble')
    device = torch.device("cpu")

print(f"SELECTED DEVICE: {device}")

Example of row:

In [ ]:
import csv
rows = []

with open(dataset_dir + "AI_scopus.csv", "r") as file:
    csvreader = csv.reader(file)
    header = next(csvreader)

    for row in csvreader:
      
      rows.append(row)
field = ['Authors','Title','Year','Source title','Volume','Issue','Art. No.','Page start','Page end','Page count','Cited by','DOI','Link','Affiliations','Authors with affiliations','Abstract','Author Keywords','Index Keywords','Correspondence Address','Editors','Publisher','ISSN','ISBN','CODEN','PubMed ID','Language of Original Document','Abbreviated Source Title','Document Type','Source','EID']
for i in range(len(rows[0])):
  print(field[i] + ": " + str(rows[438][i]))


Authors: Mallikarjuna Rao G., Kumar P., Mittal A., Vijaya Kumari G., Babu G.R.
Title: Exploration of active pixels in intelligent face recognition for constrained mobile systems
Year: 2011
Source title: Proceedings of the 5th Indian International Conference on Artificial Intelligence, IICAI 2011
Volume: 
Issue: 
Art. No.: 
Page start: 152
Page end: 166
Page count: 
Cited by: 
DOI: 
Link: https://www.scopus.com/inward/record.uri?eid=2-s2.0-84872182449&partnerID=40&md5=4d62f60078d46ac4e010bfca989dd4cb
Affiliations: Department of Computer Science Engineering, Gokaraju Rangaraju Institute of Engineering and Technology, Hyderabad, India; Computer Science Department, Jawaharlal Nehru Technological University, Hyderabad, India; Department of Computer Science Engineering, College of Engineering Roorkee, Roorkee, Uttaranchal, India
Authors with affiliations: Mallikarjuna Rao, G., Department of Computer Science Engineering, Gokaraju Rangaraju Institute of Engineering and Technology, Hyderabad, I

Install StellarGraph:

In [ ]:
import sys
if 'google.colab' in sys.modules:
  %pip install -q stellargraph[demos]==1.2.1

import stellargraph as sg

try:
    sg.utils.validate_notebook_version("1.2.1")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.2.1, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.2/435.2 KB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.3/500.3 KB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.6 MB/s eta 0:00:00


In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import itertools
import os
import csv

import matplotlib.pyplot as plt

import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, link_classification
from stellargraph.calibration import expected_calibration_error, plot_reliability_diagram
from stellargraph.calibration import IsotonicCalibration, TemperatureCalibration

from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.calibration import calibration_curve
from sklearn.isotonic import IsotonicRegression

from stellargraph import globalvar
from stellargraph import datasets
from IPython.display import display, HTML
from  stellargraph import StellarGraph
import pandas as pd


%matplotlib inline

Graph extraction block:

In [ ]:
nodes = []        #list of all nodes
nodes_feat = {}   #dictionary of all node features to use for networkx
edge_feats = {}   #dictionary of all edge features to use for networkx
source = []       #start of the edges
target = []       #end of the edges
rows = []

with open(dataset_dir + "AI_scopus.csv", "r") as file:
  csvreader = csv.reader(file)
  header = next(csvreader)
  authors = []        #list of all authors of one paper
  affiliations = []   #list of all affiliations of the authors
  m = 1
  for line in csvreader:
    print("Processing line "+ str(m))
    m += 1
    for j in range(len(line)):              #for cycle that iterates the lines of the abstract file
      tokens = nltk.word_tokenize(line[j])  #tokenize line    
      if(j == 0):                           #we are on the authors 
        author = ""
        for i in range(0,len(tokens)): 
          if(tokens[i] != ","): #ignore ","
            author = author + tokens[i]
          else:
            if(len(author) != 0):
              authors.append(author)
            author = ""
        if(len(author) != 0): #add last author, if any
          authors.append(author)
      if(j == 14):            #we are on the affiliations 
        affiliation = ""
        i = 4                 #skip name of the authors
        while i < len(tokens): 
          if(tokens[i] != ";"): #ignore ","
            affiliation = affiliation + tokens[i]
          else:
            if(len(affiliation) != 0):
              affiliations.append(affiliation)
              i += 4
            affiliation = ""
          i += 1
        if(len(affiliation) != 0): #add last affiliation, if any
          affiliations.append(affiliation)

    while(len(affiliations) < len(authors)):  #there may be a few cases where the last author has no affiliations
      affiliations.append("")
    year = line[2]
    for i in range(0, len(authors)): #add new nodes 
      if(not(authors[i] in nodes)):
        nodes.append(authors[i])
        nodes_feat.update({authors[i] : {"author_code" : hash(authors[i]), "affiliation": hash(affiliations[i])}})

    for i in range(0, len(authors)):
      for j in range(i+1, len(authors)):
        if(not((authors[i], authors[j]) in edge_feats) and not((authors[j], authors[i]) in edge_feats)): #add new edges
          source.append(authors[i])
          target.append(authors[j])
          edge_feats.update({(authors[i], authors[j]):{"year" : year, "collabs" : 1}})
        else: 
          if ((authors[i], authors[j]) in edge_feats.keys()):
            if(edge_feats.get((authors[i], authors[j])).get("year") == year):  #check if there were already collaborations between i and j this year
              n_coll = edge_feats.get((authors[i], authors[j])).get("collabs")
              edge_feats.update({(authors[i], authors[j]):{"year" : year, "collabs" : n_coll + 1}})
          if ((authors[j], authors[i]) in edge_feats.keys()):
            if(edge_feats.get((authors[j], authors[i])).get("year") == year):
              n_coll = edge_feats.get((authors[j], authors[i])).get("collabs")
              edge_feats.update({(authors[j], authors[i]):{"year" : year, "collabs" : n_coll + 1}})


author_codes = [] 
years = []
collabs = []
nodes_index = []
edges = []

for element in edge_feats:  #split edge_feats into lists to use for StellarGraph
  edges.append((element[0], element[1], dict(year=edge_feats.get(element).get("year"), collabs=edge_feats.get(element).get("collabs"))))
  years.append(int(edge_feats.get(element).get("year")))
  collabs.append(edge_feats.get(element).get("collabs"))

G = nx.MultiGraph()  #original graph
Gs = nx.MultiGraph() #graph without nodes with degree = 0

G.add_nodes_from(nodes)
nx.set_node_attributes(G, nodes_feat)
G.add_edges_from(edges)

Gs.add_nodes_from(nodes)
nx.set_node_attributes(Gs, nodes_feat)
Gs.add_edges_from(edges)


for n in G.nodes:  #remove nodes with degree = 0
  if(nx.degree(G,n) == 0):
    Gs.remove_node(n)
  else:
    nodes_index.append(n)


print("Number of connected components: " + str(nx.number_connected_components(Gs)))
node_feats = [] #list of all node features to use for StellarGraph
aff = [] #list of all author affiliations to use for StellarGraph

#list of graph features to use for StellarGraph
deg = []
har_cen = []
deg_cen = []
clos_cen = []
bet_cen = []
load_cen = []
square_clus = []
pagerank = []
constr = []

edge_load_cen = []

#graph features are calculated just once, to speed up the program
node_feats.append(nx.get_node_attributes(Gs,"author_code"))
node_feats.append(nx.get_node_attributes(Gs,"affiliation"))
node_feats.append(nx.degree(Gs))
node_feats.append(nx.harmonic_centrality(Gs))
node_feats.append(nx.degree_centrality(Gs))
node_feats.append(nx.closeness_centrality(Gs))
node_feats.append(nx.betweenness_centrality(Gs))
node_feats.append(nx.load_centrality(Gs))
node_feats.append(nx.pagerank_numpy(Gs))


dict_edge_load_cen = nx.edge_load_centrality(Gs)


for n in Gs.nodes:  #add graph features in their respective list
  author_codes.append(node_feats[0][n])
  aff.append(node_feats[1][n])
  deg.append(node_feats[2][n])
  har_cen.append(node_feats[3][n])
  deg_cen.append(node_feats[4][n])
  clos_cen.append(node_feats[5][n])
  bet_cen.append(node_feats[6][n])
  load_cen.append(node_feats[7][n])
  pagerank.append(node_feats[8][n])

for e in Gs.edges:
  edge_load_cen.append(dict_edge_load_cen[e[0:2]])

square_node_data = pd.DataFrame( #data structure for the nodes
    {"author_code": author_codes, "affiliation": aff, "degree":deg, "harmonic_centrality":har_cen, "degree_centrality":deg_cen, 
    "closeness_centrality": clos_cen, "betweenness_centrality":bet_cen, "load_centrality":load_cen,
     "pagerank": pagerank},
     index = nodes_index)

square_edges = pd.DataFrame( #data structure for the edges
    {"source": source, "target": target, "year": years, "collab": collabs, "load_centrality": edge_load_cen}
)

G_stel = StellarGraph(square_node_data, square_edges) #build graph
print(G_stel.info())

#saving the data
np.save("/content/author_codes.npy",author_codes)
np.save("/content/affiliations.npy",aff)
np.save("/content/deg.npy",deg)
np.save("/content/har_cen.npy",har_cen)
np.save("/content/deg_cen.npy",deg_cen)
np.save("/content/clos_cen.npy",clos_cen)
np.save("/content/bet_cen.npy",bet_cen)
np.save("/content/load_cen.npy",load_cen)
np.save("/content/pagerank.npy",pagerank)
np.save("/content/nodes_index.npy",nodes_index)
np.save("/content/source.npy",source)
np.save("/content/target.npy",target)
np.save("/content/years.npy",years)
np.save("/content/collabs.npy",collabs)
np.save("/content/edge_load_cen.npy",edge_load_cen)
np.save("/content/G_stel.npy",G_stel)

Processing line 1
Processing line 2
Processing line 3
Processing line 4
Processing line 5
Processing line 6
Processing line 7
Processing line 8
Processing line 9
Processing line 10
Processing line 11
Processing line 12
Processing line 13
Processing line 14
Processing line 15
Processing line 16
Processing line 17
Processing line 18
Processing line 19
Processing line 20
Processing line 21
Processing line 22
Processing line 23
Processing line 24
Processing line 25
Processing line 26
Processing line 27
Processing line 28
Processing line 29
Processing line 30
Processing line 31
Processing line 32
Processing line 33
Processing line 34
Processing line 35
Processing line 36
Processing line 37
Processing line 38
Processing line 39
Processing line 40
Processing line 41
Processing line 42
Processing line 43
Processing line 44
Processing line 45
Processing line 46
Processing line 47
Processing line 48
Processing line 49
Processing line 50
Processing line 51
Processing line 52
Processing line 53
Pr